## Численные методы. Задание 2.1
#### Козловский А.А., гр. 2253

In [38]:
import numpy as np

### Определения класса для решения СЛАУ методом прогонки
#### Теоретические заметки

Метод прогонки состоит из двух частей: прямой ход и обратный. Во время прямого хода, мы считаем дополнительные коэффициенты, которые понадобяться нам на обратном ходу, чтобы рекурсивно посчитать значения переменных, являющиеся решением.


Допустим матрица коэффициентов СЛАУ задана в следующем виде:
$$\begin{pmatrix}
b_{0} & c_{0} & 0 & 0 & ... & 0 & 0  & 0\\
a_{1} & b_{1} & c_{1} & 0 & ... & 0 & 0 & 0\\
0 & a_{2} & b_{2} & c_{2} & ... & 0 & 0 & 0 \\
&&&&{...} \\
0 & 0 & 0 & 0 & ... & 0 & a_{n} & b_{n} 
\end{pmatrix}$$

Тогда, вектора вспомогательных коэффициентов $\vec{r}$ и $\vec{s}$ будут находиться по следующим формулам:
$$\begin{cases}
r_{2} = \frac{-c_{1}}{b_{1}} \\
s_{2} = \frac{f_{1}}{b_{1}}
\end{cases}$$
<p>
$$\begin{cases}
r_{i+1} = \frac{-c_{i}}{a_{i}r_{i} + s_{i}} \\
s_{i+1} = \frac{f_{i}-a_{i}b_{i}}{a_{i}r_{i} + b_{i}}
\end{cases}$$
    
После векторов вспомогательных коэффициентов найдем, благодаря ним решение по следующим формулам:
    
$$x_{n} = \frac{f_{n} - a_{n}s_{n}}{b_{n} + a_{n}r_{n}}$$
$$x_{i} = r_{i+1}x_{i+1} + s_{i+1}, i=\overline{n-1, 1}$$

#### Реализация

In [202]:
class Solver:
    def __init__(self, matrix, free):
        self.matrix = matrix
        self.free = free
        self.result = None
        
    def _is_satisfied(self, k):
        """Метод для проверки выполнения условия трехдиагональной
        или пятидиагональной матрицы"""
        assert k == 3 or k == 5
        
        for i, row in enumerate(matrix):
            for j, el in enumerate(row):
                if j > i + k // 2 and el != 0:
                    return False
                if j < i - k // 2 and el != 0:
                    return False
                
        return True
    
    def _thomas_forward_pass(self, matrix):
        """Метод для прямого прохода для метода прогонки
        matrix: расширенная матрица СЛАУ
        returns: два массива коэффициентов для обратного хода
        метода прогонки"""
        
        n = matrix.shape[0]
        r, s = np.zeros((n)), np.zeros((n))
        b, c, f = matrix[0, 0], matrix[0, 1], matrix[0, n]
        r[1], s[1] = -c / b, f / b
        
        for i in range(2, n):
            a, b, c, f = *matrix[i-1, i-2:i+1], matrix[i-1, n]
            bottom = a * r[i - 1] + b
            r[i] = -c / bottom
            s[i] = (f - a * s[i - 1]) / bottom
            
        return r, s
    
    def _thomas_backward_pass(self, matrix, r, s):
        """Метод для обратного хода метода прогонки
        matrix: расширенная матрица системы
        r: массив коэффициентов alpha
        s: массив коэффициентов betta
        returns: вектор значений x"""
        n = self.matrix.shape[0]
        x = np.zeros((n))
        
        a, b, f = matrix[n-1, n-2:]
        x[n - 1] = (f - a * s[n - 1]) / (b + a * r[n - 1])
        
        for i in range(n - 2, -1, -1):
            x[i] = r[i + 1] * x[i + 1] + s[i + 1]
            
        return x

    def thomas(self):
        """Метод прогонки
        returns: вектор значений x"""
        assert self._is_satisfied(3)
        matrix = np.concatenate((self.matrix, np.array([self.free]).T), axis=1)  
        n = matrix.shape[0]
        
        r, s = self._thomas_forward_pass(matrix)
        self.result = self._thomas_backward_pass(matrix, r, s)
        return self.result
    
    def residual_count(self):
        """Метод для подсчёта невязки
        returns: невязку получившегося решения"""
        assert self.result is not None
        matrix, result = self.matrix, self.result
        
        actual_free = np.array([np.dot(row, result) for row in matrix])
        return np.linalg.norm(actual_free - free)
        

### Тестирование класса
#### Генерация трехдиагональной матрицы и вектора свободных переменных

In [203]:
matrix = np.array([[np.random.rand() * 10 if i - 1 <= j <= i + 1
            else 0
          for j in range(10)]
          for i in range(10)])

In [204]:
print(np.round(matrix, 2))

[[8.42 6.23 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [4.73 9.11 1.43 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   9.65 1.07 0.08 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   4.04 2.51 7.28 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   8.08 3.13 0.97 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   6.83 2.61 9.92 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   3.1  1.78 5.95 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   2.18 5.2  8.75 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.06 4.03 6.13]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.61 7.71]]


In [205]:
free = np.random.rand((10)) * 10
print(free)

[6.56234972 8.58259754 6.62165849 5.57398924 6.04732614 1.76661002
 2.92564067 8.14115724 8.28554603 8.77162054]


#### Решение СЛАУ и анализ результатов

In [206]:
s = Solver(matrix, free)

In [207]:
result = s.thomas()

In [208]:
s.residual_count()

3.794299872214038e-15